In [1]:
from extract_desserts import *
from pattern.text.en import singularize
from nltk import FreqDist
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(seed=2019)

import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    datefmt='%H:%M:%S',
                   level=logging.INFO)

from time import time
import multiprocessing

import warnings
warnings.filterwarnings('ignore')

In [2]:
#load the raw data
# lets use the default value that only loads 250 recipes,
# instead of 1 million
raw_data = get_raw_data()
raw_ingredients = get_raw_ingredients()

Loading file  ../data/sample_layer1.json
Loading ingredient file: ../data/sample_det_ingrs.json


In [3]:
print('N recipes: {}\t N list of ingredients: {}'.format(len(raw_data),
      len(raw_ingredients)))


N recipes: 20000	 N list of ingredients: 20000


In [4]:
## get a list of desserts only
desserts, dessert_ings = find_desserts(raw_data, raw_ingredients)#,test_id='02916fd389')


In [9]:
total_recipes = len(raw_data)
dessert_recipes = len(desserts)
dessert_ingredients = len(dessert_ings)
print('Will look at {} dessert recipes, out of {} (~{:1.1f}%)'.format(
                            dessert_recipes, total_recipes,
                            dessert_recipes/total_recipes*100))

Will look at 3848 dessert recipes, out of 20000 (~19.2%)


In [45]:
not_a_flavor_list = ['flour','cake mix','baking soda','baking powder', 'canola oil',
                     'vegetable oil','cornstarch','shortening','margarine','yeast',
                     'gelatin' ,'coloring','corn syrup','cooking spray', 
         'crisco', 'cake', 'crisco','xanthan','bicarbonate','pie shell','pie crust',
                     'cornmeal','splenda','stevium','ice cube'
                    ]

generic_ingredient_list = ['pumpkin', 'vanilla','cinnammon','cocoa powder',
                           'chocolate chip','salted butter', 'graham cracker','mint',
                          'salted butter']



In [46]:
import re


def in_not_a_flavor_list(ingredient):
    for word in not_a_flavor_list:
        if re.search(word, ingredient):
            return True   
    return False


def is_generic_ingredient(ingredient):
    for word in generic_ingredient_list:
#         print(word,ingredient)
        if re.search(word, ingredient):
            return True, word
    return False, ingredient

def clean_dessert_ingredients(all_ingredients):
    count =0
    for item, ingredients_list in enumerate(all_ingredients):
        to_remove = []
        for ingr_item, ingredient in enumerate(ingredients_list['ingredients']):
            tmp_ingredient = ingredient['text'].lower().replace(' - ','-')
            tmp_ingredient = tmp_ingredient.replace("'",'')
            tmp_ingredient = tmp_ingredient.replace(" & ",'&')
            tmp_ingredient = tmp_ingredient.replace('fat free', 'fat-free')
#             tmp_ingredient = singularize(tmp_ingredient)
            
#             print(ingredient['text'].replace(' - ','-'))
            if re.search('water', tmp_ingredient) and not re.search('watermelon',tmp_ingredient) and not re.search('rose water',tmp_ingredient):
                to_remove.append(ingr_item)
            
            if in_not_a_flavor_list(tmp_ingredient):
                to_remove.append(ingr_item)
            elif re.match('oil', tmp_ingredient):
                to_remove.append(ingr_item)

                        
            
            if re.search('heavy', tmp_ingredient) and re.search('cream',tmp_ingredient):
                tmp_ingredient='heavy cream'
            
            generic_ingr, ingr = is_generic_ingredient(tmp_ingredient)
            if generic_ingr:
                tmp_ingredient = ingr

                
#             elif (re.search('purpose flour', tmp_ingredient) 
#                   or re.search('cake flour',tmp_ingredient) 
#                   or re.search('rising flour', tmp_ingredient) 
#                   or re.search('plain flour', tmp_ingredient)
#                   or re.search('white flour', tmp_ingredient)
#                   or re.search('raising flour', tmp_ingredient)

#                  ):
#                 tmp_ingredient = 'flour' 
            elif re.search('brown sugar', tmp_ingredient) or re.search('demerara sugar', tmp_ingredient):
                tmp_ingredient = 'brown sugar'
            
            elif re.search('powdered sugar', tmp_ingredient) or re.search('confectioners sugar',tmp_ingredient) :
                tmp_ingredient ='powdered sugar'
            elif re.search('sugar', tmp_ingredient) or re.search('white sugar', tmp_ingredient) or re.search('granulated sugar',tmp_ingredient):
                tmp_ingredient = 'sugar'

            elif re.search('semi-sweet', tmp_ingredient) and re.search('chocolate',tmp_ingredient):
                tmp_ingredient = 'chocolate'

            if (not re.search('flour', tmp_ingredient) 
                and not re.search('molasses',tmp_ingredient)
                and not re.search('oats',tmp_ingredient)):
                tmp_ingredient = singularize(tmp_ingredient)


            all_ingredients[item]['ingredients'][ingr_item]['text'] = tmp_ingredient
            
        if len(to_remove)>0:
            
            to_remove.sort(reverse=True)
            
                
#             print(item,'to remove', to_remove)
            for i in to_remove:
#                 print('try to remove', all_ingredients[item]['ingredients'][i])
                del all_ingredients[item]['ingredients'][i]
#         for iremove in to_remove:
#             del all_ingredients
#         print('--------------------------')
        if item%500==0:
            logging.info("read {0} recipes".format(item))
    return all_ingredients

def print_dessert_ingredients(all_ingredients):
    count =0
    for item, ingredients_list in enumerate(all_ingredients):
        for ingr_item, ingredient in enumerate(ingredients_list['ingredients']):
            
            
            print(ingredient['text'])
        
        print('--------------------------')
        if item==50:
            break
    return all_ingredients

In [ ]:
clean_ingredients =  clean_dessert_ingredients(dessert_ings);
# print_dessert_ingredients(clean_ingredients)
# print(dessert_ings)


In [37]:
len(clean_ingredients)
# item = np.random.randint(dessert_recipes)
# print(item, dessert_ings[item]['valid'])
# dessert_ings[item]['ingredients']

3848

In [ ]:
m = re.match('onion', 'yellow onions' )
print(m)

m = re.search('onion', 'yellow onions' )
print(m)

# Let's look at the unique words

In [ ]:
item=0
len(dessert_ings),len(dessert_ings[item]['ingredients'])

## Lets create a list of ingredients
And we will us it as an input to word2Vec

In [ ]:
ingredients_per_recipe = []
count =0
for drecipe in dessert_ings:
    ings =[]
    for entry in drecipe['ingredients']:
        ings.append(entry['text'])
    ingredients_per_recipe.append(ings)
    count += 1
    if count%500==0:
        logging.info("read {0} ingredients".format(count))

#         break;

# print(all_dingredients)

In [ ]:
ingredients_per_recipe[0]

## Create a Word2Vec model training on the ingredients. 


In [ ]:
vec_size = 100;
model = gensim.models.Word2Vec(
   # all_dingredients,
    size=vec_size,
    window=5,
    min_count=20,
    workers=10,
    alpha=0.02,
    iter=4,
    sg=0)

In [ ]:
t = time()
model.build_vocab(ingredients_per_recipe, progress_per=1000)

In [ ]:
model.train(ingredients_per_recipe, total_examples=model.corpus_count,
           epochs=40, report_delay=1)

In [ ]:
model.init_sims(replace=True)

In [ ]:
w1='almond'
w1='rum'
model.wv.most_similar(positive=w1, topn=4)

In [ ]:
w1='rum'
model.wv.most_similar(positive=['apple','banana'],
                      negative=['rum'],topn=10)

In [ ]:
# witem = model.wv.__getitem__(['ginger'])
# witem[0]

In [ ]:
model.wv.doesnt_match(['apple','butter','banana',
                       'rum','flour'])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# def ingredient_scatterplot(model, word, list_names):
def ingredient_scatterplot(model, word):
    # create an array with the same size as the vector
    arrays = np.empty((0,vec_size), dtype='f')
    # set the original word,     
    word_labels = [word]
    color_list = ['red']
    # append the vector for the test word
    arrays = np.append(arrays, model.wv.__getitem__([word]),axis=0)
    
    ####### get the closest words and make then blue
    #get a list of the most similar words
    close_words = model.wv.most_similar([word],topn=25)
    
    for word_score in close_words:
        # for each word get the vector  
        word_vector = model.wv.__getitem__([word_score[0]])
        # get the name
        word_labels.append(word_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, word_vector, axis=0)
             
    Y = PCA(n_components=2).fit_transform(arrays)
    
#     np.set_printoptions(suppress=True)
    
#     Y = TSNE(n_components=2, random_state=0, 
#              perplexity=15).fit_transform(reduc)
    df = pd.DataFrame({'x':[x for x in Y[:, 0]],
                       'y':[y for y in Y[:, 1]],
#                        'z':[z for z in Y[:, 2]],
                       'words':word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(8,6)
    
    p1 = sns.regplot(data=df, 
                    x='x',y='y',fit_reg=False,
                    marker='x',
                    scatter_kws={'s':40,
                                'facecolors':df['color']})
    
    for line in range(df.shape[0]):
        p1.text(df['x'][line],df['y'][line],#df['z'][line],
               '  '+df['words'][line].title(),
               horizontalalignment='left',
                verticalalignment='bottom',size='large',
                color=df['color'][line],weight='normal'
               ).set_size(10)
#     fig = plt.figure(figsize=(8,6))
#     ax = Axes3D(fig)  
    
#     ax.scatter(df['x'], df['y'], df['z'], marker='o')
    plt.xlim(Y[:,0].min()-0.1, Y[:,0].max()+0.1 )
    plt.ylim(Y[:,1].min()-0.1, Y[:,1].max()+0.1 )
#     plt.zlim(Y[:,2].min(), Y[:,2].max() )
    plt.title('Food pairing for {}'.format(word.title()))

In [ ]:
ingredient_scatterplot(model, 'apple')

In [ ]:
test_word = 'rum'

In [ ]:
ingredient_scatterplot(model, test_word)



### Make a network graph

In [ ]:
import networkx as nx


In [ ]:

w1='rum'
ingred_list = model.wv.most_similar(positive=w1, topn=20)

# print(ingred_list)
G = nx.Graph()
G.add_node(w1,nodesize=20)
for n_node, ingred in enumerate(ingred_list):
    G.add_node(ingred[0], nodesize=ingred[1]*10)
    G.add_weighted_edges_from([(w1, ingred[0], ingred[1])])
               
sizes = [G.node[node]['nodesize']*300 for node in G]
        #     G.add_edge(1,n_node+2)
# G.add_nodes_from([1,2,3,4,5,6,7])
# G.add_edge(1,2)
# G.add_edge(1,3)
# G.add_edge(1,4)
# G.add_edge(1,5)
# G.add_edge(1,6)
# G.add_edge(1,7)

# G.node[1]['label']='blue'
# G.add_edge(1,4)

plt.figure(figsize=(10,8))

options = {
    'edge_color': '#4C34EB',
    'width': 1,
    'with_labels': True,
    'font_weight': 'heavy',
}

nx.draw(G,  node_size=sizes, 
        pos=nx.spring_layout(G, k=0.5, iterations=50), **options)
# ax = plt.gca()
# ax.collections[0].set_edgecolor('#5555')
# plt.show()

In [ ]:
G.nodes

## look at unique values

In [38]:
# create bag of ingredients

all_ingredients = []

for recipe_ings in clean_ingredients:# dessert_ings:
    n_ingredients = len(recipe_ings['valid'])
    for item, ingredient in enumerate(recipe_ings['ingredients']):
#         if recipe_ings['valid'][item] ==True:
        ingred = ingredient['text'].lower()

        all_ingredients.append(ingred)
                

            

In [39]:

### Funciton to retrieve reciepes given an ingredient list

def get_recipes_for_ingredients(search_list, ingredients_list, recipe_list):
    """
    Function to look for a recipe with all the ingredients in  the list passed
    
    Output:
        List of ingredients.
    """
    
    found_recipe_list = []
    found_ingredient_list = []
    for item,recipe_ingrs in enumerate(ingredients_list):
        temp_ingr_list = []
        ingredients = recipe_ingrs['ingredients']
        n_ingredients = len(ingredients)
        found_ingredient =0
        for ingredient in ingredients:
            if re.search(search_list[0], ingredient['text']):
                found_ingredient=1
            temp_ingr_list.append(ingredient['text'])

        # find all elements in the list
#         result = all(elem in temp_ingr_list for elem in search_list)
        # find any elements in the list
        result = any(singularize(elem) in temp_ingr_list for elem in search_list)
        
        
        if result or found_ingredient==1:
            found_recipe_list.append(recipe_list[item])
            found_ingredient_list.append(temp_ingr_list)
#             print('adding item', item)
           
    
    
    
    return found_recipe_list, found_ingredient_list
    

In [17]:
# all_ingredients

In [40]:
ingredient_set = set(all_ingredients)
print('with {} recipes, there are {} ingredients, and {} unique ingredients.'.format(
                dessert_recipes, len(all_ingredients), len(ingredient_set)))

with 3848 recipes, there are 27625 ingredients, and 2179 unique ingredients.


In [68]:
ingredient_set = set(all_ingredients)
print('with {} recipes, there are {} ingredients, and {} unique ingredients.'.format(
                dessert_recipes, len(all_ingredients), len(ingredient_set)))

with 3848 recipes, there are 25197 ingredients, and 2136 unique ingredients.


In [41]:
fdist = FreqDist(all_ingredients)
print(fdist)

<FreqDist with 2179 samples and 27625 outcomes>


In [42]:
common = fdist.most_common(2500)

In [43]:
# common
# f= open("words.txt","w+")
for cc in common:
    if re.search('pie', cc[0]) and cc[1]>10:
        print(cc)
#     f.write('{}: \t {}\n'.format(cc[0], cc[1]))

('cherry pie filling', 16)
('toffee piece', 12)


In [44]:
common

[('sugar', 2867),
 ('vanilla', 2066),
 ('egg', 1853),
 ('salt', 1659),
 ('butter', 1513),
 ('brown sugar', 849),
 ('milk', 741),
 ('salted butter', 632),
 ('powdered sugar', 564),
 ('chocolate chip', 450),
 ('cinnamon', 444),
 ('heavy cream', 355),
 ('ground cinnamon', 338),
 ('egg yolk', 297),
 ('cream cheese', 290),
 ('pecan', 267),
 ('cocoa powder', 243),
 ('honey', 237),
 ('walnut', 236),
 ('egg white', 208),
 ('banana', 199),
 ('nutmeg', 176),
 ('pumpkin', 176),
 ('buttermilk', 173),
 ('lemon juice', 171),
 ('sour cream', 154),
 ('whipping cream', 141),
 ('raisin', 123),
 ('ground nutmeg', 117),
 ('almond extract', 110),
 ('nut', 107),
 ('ground ginger', 103),
 ('peanut butter', 101),
 ('cocoa', 101),
 ('ground clove', 96),
 ('apple', 95),
 ('whole milk', 95),
 ('lemon', 90),
 ('chocolate', 87),
 ('strawberry', 85),
 ('low-fat milk', 84),
 ('sweetened condensed milk', 84),
 ('orange juice', 82),
 ('evaporated milk', 82),
 ('semisweet chocolate', 81),
 ('bittersweet chocolate', 79)

In [33]:
found_lists, found_ingredients = get_recipes_for_ingredients(['pasta'], dessert_ings,desserts)
print('found {} recipes'.format(len(found_lists)))

for item, rec in enumerate(found_lists):
    print('{}\t{}'.format(item, rec['title']))

found 1 recipes
0	Chocolate Stuffed Shells


In [ ]:
item = 0
found_lists[item],found_ingredients[item]

In [ ]:
test = 'onio'

for ingr in found_ingredients[82]:
    if re.match(test, ingr):
        print('----> found a',test, ingr)
    else:
        print(ingr)

In [ ]:
test_ingredients = []
for ingred in dessert_ings[2029]['ingredients']:
    test_ingredients.append(ingred['text'])
print(test_ingredients)

In [ ]:
food_pattern = 'flourish'
# for ing in test_ingredients:
m = re.search(food_pattern, 'vanilla and flour don')
print(m)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(12, 8))
# plt.ylim(0,120)
fdist.plot(50,cumulative=False)
# plt.ylim(0,30)
# plt.show()

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
np.random.seed(23)

In [ ]:
stopwords_list  = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list +=

In [ ]:
stopwords_list[-20:]

In [ ]:
# import spacy

In [ ]:
# !python -m spacy download en

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline
import spacy

In [ ]:
nlp = spacy.load('en')

In [ ]:
desserts[3204]['instructions']

test_instructions = ''

for instr in desserts[3204]['instructions']:
    test_instructions += instr['text'] + ' '
    
print(test_instructions)

In [ ]:
parsed_text = nlp(test_instructions)

In [ ]:
parsed_text

In [ ]:
for num, entity in enumerate(nlp(test_instructions).ents):
    print(num+1,'Entity: ',entity, entity.label_)

In [ ]:
token_pos = [token.pos for token in nlp(test_instructions)]
tokens = [token for token in nlp(test_instructions)]
sd = list(zip(tokens, token_pos))
sd = pd.DataFrame(sd, columns=['token','pos'])
sd.head()

In [ ]:
sd

In [ ]:
max_num_words = 1000
max_seg_length = 100
tokenizer = Tokenizer(num_words=max_num_words)